In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index


array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    # ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA" and attack_name!= "AutoPGD":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
x_test_adv_0_1.shape

(797797, 56)

In [6]:
from xgboost import XGBClassifier

print('xgb')
xgb100 = XGBClassifier()
xgb100.fit(x_test_adv_0_1, y_label_0_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf100 = RandomForestClassifier()
rf100.fit(x_test_adv_0_1, y_label_0_1)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt100 = DecisionTreeClassifier()
dt100.fit(x_test_adv_0_1, y_label_0_1)

xgb
RF
DT


DecisionTreeClassifier()

In [7]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [8]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb100.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Exclude2Attack/UNSW_Input100/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Exclude2Attack/UNSW_Input100/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [   139    240   2156   2061     34    178      1     23 108776    269
     94]
(139, 56) (139,)
Save 0 to baseline_Def1.npy
(240, 56) (240,)
Save 1 to baseline_Def2.npy
(2156, 56) (2156,)
Save 2 to baseline_Def3.npy
(2061, 56) (2061,)
Save 3 to baseline_Def4.npy
(34, 56) (34,)
Save 4 to baseline_Def5.npy
(178, 56) (178,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(23, 56) (23,)
Save 7 to baseline_Def8.npy
(108776, 56) (108776,)
Save 8 to baseline_Def9.npy
(269, 56) (269,)
Save 9 to baseline_Def10.npy
(94, 56) (94,)
Save 10 to baseline_Def11.npy
Execution Time: 1.254035 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [ 85994    145   1428    740    756 148980      1    612 103199     25
     33]
(85994, 56) (85994,)
Save 0 to BIM_Def1.npy
(145, 56) (145,)
Save 1 to BIM_Def2.npy
(1428, 56) (1428,)
Save 2 to BIM_Def3.npy
(740, 56) (740,)
Save 3 to BIM_Def4.npy
(756, 56) (756,)
Save 4 to BIM_Def5.npy
(148980, 56) 

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf100.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input100/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input100/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [   473    508   2534   2047     90   1207      1    327 106355    284
    145]
(473, 56) (473,)
Save 0 to baseline_Def1.npy
(508, 56) (508,)
Save 1 to baseline_Def2.npy
(2534, 56) (2534,)
Save 2 to baseline_Def3.npy
(2047, 56) (2047,)
Save 3 to baseline_Def4.npy
(90, 56) (90,)
Save 4 to baseline_Def5.npy
(1207, 56) (1207,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(327, 56) (327,)
Save 7 to baseline_Def8.npy
(106355, 56) (106355,)
Save 8 to baseline_Def9.npy
(284, 56) (284,)
Save 9 to baseline_Def10.npy
(145, 56) (145,)
Save 10 to baseline_Def11.npy
Execution Time: 2.152218 seconds
(341913,)
[0 1 2 3 4 5 7 8 9] [ 52724      3   1017    388      3 175109     38 112626      5]
(52724, 56) (52724,)
Save 0 to BIM_Def1.npy
(3, 56) (3,)
Save 1 to BIM_Def2.npy
(1017, 56) (1017,)
Save 2 to BIM_Def3.npy
(388, 56) (388,)
Save 3 to BIM_Def4.npy
(3, 56) (3,)
Save 4 to BIM_Def5.npy
(175109, 56) (175109,)
Save 5 to BIM_Def6.npy

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt100.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Exclude2Attack/UNSW_Input100/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Exclude2Attack/UNSW_Input100/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [   492    554   2701   2127    100   1302      1    335 105877    307
    175]
(492, 56) (492,)
Save 0 to baseline_Def1.npy
(554, 56) (554,)
Save 1 to baseline_Def2.npy
(2701, 56) (2701,)
Save 2 to baseline_Def3.npy
(2127, 56) (2127,)
Save 3 to baseline_Def4.npy
(100, 56) (100,)
Save 4 to baseline_Def5.npy
(1302, 56) (1302,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(335, 56) (335,)
Save 7 to baseline_Def8.npy
(105877, 56) (105877,)
Save 8 to baseline_Def9.npy
(307, 56) (307,)
Save 9 to baseline_Def10.npy
(175, 56) (175,)
Save 10 to baseline_Def11.npy
Execution Time: 0.875918 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [ 90167   2415   5488   4468   2650 148662    104   6082  80860    708
    309]
(90167, 56) (90167,)
Save 0 to BIM_Def1.npy
(2415, 56) (2415,)
Save 1 to BIM_Def2.npy
(5488, 56) (5488,)
Save 2 to BIM_Def3.npy
(4468, 56) (4468,)
Save 3 to BIM_Def4.npy
(2650, 56) (2650,)
Save 4 to BIM_Def5.npy

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}100")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_Exclude2Attack/UNSW_Input100/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input100_Exclude2Attack.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB100,139,240,2156,2061,34,178,1,23,108776,269,94,113971
1,XGB100,85994,145,1428,740,756,148980,1,612,103199,25,33,341913
2,XGB100,60299,80,1416,906,265,175201,1,1767,101941,12,25,341913
3,XGB100,85994,145,1428,740,756,148980,1,612,103199,25,33,341913
4,XGB100,74343,689,2374,353,219,243365,278,18173,1671,308,140,341913
5,XGB100,80198,603,4456,4536,689,152991,1,634,96950,610,245,341913
6,XGB100,466,689,6389,5563,103,572,3,69,327093,719,247,341913
7,XGB100,26781,24,753,1753,13,77,1,2,312466,25,18,341913
8,XGB100,90322,389,574,734,685,140734,1,166,108283,17,8,341913
9,XGB100,63625,107,1507,1104,689,169372,1,59,105414,15,20,341913
